In [8]:
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import pandas as pd
import importlib
import sys

# Determine root directory - go up from notebooks folder to portfolio_analyser
if Path.cwd().name == 'notebooks':
    ROOT = Path.cwd().parent
else:
    ROOT = Path.cwd()

src_path = ROOT / 'src'

# Add src to path if not already there
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import analysis and reporting functions
import technical_analysis as ta
import portfolio_reports as pr
from portfolio_reports import create_trading_playbook_pdf, create_portfolio_tracker_excel, cleanup_old_reports

# Reload modules to pick up any code changes (uncomment during development)
importlib.reload(ta)
importlib.reload(pr)

# Setup
RESULTS_DIR = ROOT / 'portfolio_results'
RESULTS_DIR.mkdir(exist_ok=True)
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')

In [9]:
# ===================================================================================================
# CELL 2: LOAD PORTFOLIO DATA
# ===================================================================================================

# Load holdings and targets from data folder
holdings_file = ROOT / 'data' / 'holdings.csv'
targets_file = ROOT / 'data' / 'targets.csv'
stocks_file = ROOT / 'data' / 'stocks.txt'

# Read holdings
if not holdings_file.exists():
    print(f"Creating new holdings file: {holdings_file}")
    holdings_df = pd.DataFrame(columns=['ticker', 'quantity', 'avg_cost'])
    holdings_df.to_csv(holdings_file, index=False)
else:
    holdings_df = pd.read_csv(holdings_file)

# Read targets
if not targets_file.exists():
    print(f"Error: {targets_file} not found")
    raise FileNotFoundError(f"targets.csv required")

targets_df = pd.read_csv(targets_file)

# Convert target_pct from 0-100 to 0-1 if needed
if targets_df['target_pct'].max() > 1.0:
    targets_df['target_pct'] = targets_df['target_pct'] / 100.0

# Get list of individual tickers (no baskets)
baskets = {
    'BTC': ['BTC-USD']
}

individual_tickers = []
for ticker in targets_df['ticker']:
    if ticker in baskets:
        individual_tickers.extend(baskets[ticker])
    else:
        individual_tickers.append(ticker)

print(f"✓ Loaded {len(holdings_df)} holdings")
print(f"✓ Loaded {len(targets_df)} target allocations")
print(f"✓ Expanded to {len(individual_tickers)} individual tickers")
print(f"\nHoldings:\n{holdings_df.to_string()}")

✓ Loaded 9 holdings
✓ Loaded 7 target allocations
✓ Expanded to 7 individual tickers

Holdings:
    ticker  quantity  avg_cost last_updated  min_quantity
0     TSLA      72.0     310.6   2026-01-04           0.0
1     NVDA       0.0       0.0          NaN           0.0
2     ASML       0.0       0.0          NaN           0.0
3      AMD       0.0       0.0          NaN           0.0
4     AVGO       0.0       0.0          NaN           0.0
5     ALAB       0.0       0.0          NaN           0.0
6     MRVL       0.0       0.0          NaN           0.0
7  BTC-USD       0.5   50090.1   2026-01-04           0.5
8  SOL-USD       0.0       0.0          NaN           0.0


In [10]:
# Set actual cash available
CASH_AVAILABLE = 110439.86

# Calculate initial portfolio total from holdings at avg_cost
PORTFOLIO_TOTAL = 0
initial_holdings_value = 0

for _, row in holdings_df.iterrows():
    quantity = row['quantity']
    if quantity > 0:
        initial_holdings_value += quantity * row['avg_cost']

PORTFOLIO_TOTAL = initial_holdings_value

print(f"Cash Available: ${CASH_AVAILABLE:,.2f}")
print(f"Initial Holdings Value (avg cost): ${initial_holdings_value:,.2f}")
print(f"Initial Portfolio Total (avg cost): ${PORTFOLIO_TOTAL:,.2f}")
print(f"Total Portfolio Value (avg cost + cash): ${PORTFOLIO_TOTAL + CASH_AVAILABLE:,.2f}")
print(f"\nNote: Target values will be calculated after fetching current market prices")

Cash Available: $110,439.86
Initial Holdings Value (avg cost): $47,408.25
Initial Portfolio Total (avg cost): $47,408.25
Total Portfolio Value (avg cost + cash): $157,848.11

Note: Target values will be calculated after fetching current market prices


In [11]:
# ===================================================================================================
# CELL 3: ANALYZE PORTFOLIO STOCKS
# ===================================================================================================

print("Starting portfolio analysis...")
print(f"Stocks to analyze: {len(individual_tickers)}")
print(f"Concurrency: 2 threads")
print(f"Cache: 24hr TTL, 1-2s delay between API calls\n")

results = []
errors = []

def analyze_stock(ticker):
    """Analyze single stock with error handling"""
    try:
        result = ta.analyze_ticker(ticker)
        if result:
            return ticker, result, None
    except Exception as e:
        return ticker, None, str(e)
    return ticker, None, "Unknown error"

with ThreadPoolExecutor(max_workers=2) as executor:
    futures = {executor.submit(analyze_stock, ticker): ticker for ticker in individual_tickers}

    for future in as_completed(futures):
        ticker, result, error = future.result()

        if result and 'error' not in result:
            results.append(result)
            print(f"✓ {ticker}: {result['signal']}")
        else:
            error_msg = result.get('error', error) if result else error
            errors.append((ticker, error_msg))
            print(f"✗ {ticker}: {error_msg}")

print(f"\n{'='*80}")

print(f"Analysis complete: {len(results)} success, {len(errors)} errors")
print(f"{'='*80}")

Starting portfolio analysis...
Stocks to analyze: 7
Concurrency: 2 threads
Cache: 24hr TTL, 1-2s delay between API calls

✓ TSLA: FULL HOLD + ADD
✓ NVDA: FULL HOLD + ADD
✓ ASML: FULL HOLD + ADD
✓ AVGO: HOLD
✓ AMD: HOLD
✓ ALAB: HOLD MOST + REDUCE
✓ MRVL: FULL HOLD + ADD

Analysis complete: 7 success, 0 errors


In [12]:
# Recalculate PORTFOLIO_TOTAL using current prices from results
PORTFOLIO_TOTAL_CURRENT = 0

print("\nRecalculating portfolio with current prices:")
for _, row in holdings_df.iterrows():
    ticker = row['ticker']
    quantity = row['quantity']
    if quantity > 0:
        # Find the result for this ticker
        result = next((r for r in results if r['ticker'] == ticker), None)
        if result:
            current_price = result['current_price']
            current_value = quantity * current_price
            PORTFOLIO_TOTAL_CURRENT += current_value
            print(f"{ticker}: {quantity:.4f} quantity @ ${current_price:.2f} = ${current_value:,.2f}")

PORTFOLIO_TOTAL = PORTFOLIO_TOTAL_CURRENT
total_portfolio_value = PORTFOLIO_TOTAL + CASH_AVAILABLE

print(f"\n✓ Updated PORTFOLIO_TOTAL (current prices) = ${PORTFOLIO_TOTAL:,.2f}")
print(f"✓ Total Portfolio Value (current + cash) = ${total_portfolio_value:,.2f}")

# NOW calculate target_value based on CURRENT portfolio value
print(f"\nAuto-calculating target values based on CURRENT portfolio value ${total_portfolio_value:,.2f}...")
targets_df['target_value'] = (targets_df['target_pct'] * total_portfolio_value).round(0).astype(int)

# Save updated targets back to CSV
targets_df.to_csv(targets_file, index=False)
print(f"✓ Updated target values in {targets_file.name}")

# Display updated targets
print(f"\nUpdated Target Allocations (based on current prices):")
for _, row in targets_df.iterrows():
    print(f"  {row['ticker']:<10} {row['target_pct']*100:>3.0f}% → ${row['target_value']:>10,}")

total_target_value = targets_df['target_value'].sum()
print(f"\nTotal Target Value: ${total_target_value:,.2f} ({total_target_value/total_portfolio_value*100:.1f}%)")


Recalculating portfolio with current prices:
TSLA: 72.0000 quantity @ $436.52 = $31,429.41

✓ Updated PORTFOLIO_TOTAL (current prices) = $31,429.41
✓ Total Portfolio Value (current + cash) = $141,869.27

Auto-calculating target values based on CURRENT portfolio value $141,869.27...
✓ Updated target values in targets.csv

Updated Target Allocations (based on current prices):
  TSLA        50% → $    70,935
  NVDA        10% → $    14,187
  ASML        10% → $    14,187
  AMD         10% → $    14,187
  AVGO        10% → $    14,187
  ALAB         5% → $     7,093
  MRVL         5% → $     7,093

Total Target Value: $141,869.00 (100.0%)


In [13]:
# ===================================================================================================
# CELL 5: BUILD PORTFOLIO POSITIONS
# ===================================================================================================

portfolio_positions = []
buy_count = 0
sell_count = 0
hold_count = 0

for result in results:
    ticker = result['ticker']

    # Find target allocation
    target = targets_df[targets_df['ticker'] == ticker]
    if target.empty:
        print(f"Warning: No target allocation for {ticker}")
        continue

    target_pct = target.iloc[0]['target_pct']

    # Find holding
    holding = holdings_df[holdings_df['ticker'] == ticker]
    quantity = holding.iloc[0]['quantity'] if not holding.empty else 0

    # Calculate current value and position metrics
    current_price = result['current_price']
    current_value = quantity * current_price

    # Calculate min quantity (not tradeable below this)
    min_quantity = holding.iloc[0].get('min_quantity', 0) if not holding.empty else 0
    min_value = min_quantity * current_price
    tradeable_quantity = max(0, quantity - min_quantity)
    tradeable_value = tradeable_quantity * current_price

    # Calculate position gap
    total_portfolio_value = PORTFOLIO_TOTAL + CASH_AVAILABLE
    current_pct = (current_value / total_portfolio_value) if total_portfolio_value > 0 else 0
    gap_value = (target_pct * total_portfolio_value) - current_value

    # Position gap structure for MA feasibility checks
    position_gap = {
        'gap_value': gap_value,
        'target_pct': target_pct,
        'current_pct': current_pct
    }

    # Check if MAs block path to resistance
    adjusted_r1, adjusted_r2, adjusted_r3, ma_note = ta.adjust_sell_levels_for_mas(
        d50=result.get('d50'),
        d100=result.get('d100'),
        d200=result.get('d200'),
        r1=result.get('r1'),
        r2=result.get('r2'),
        r3=result.get('r3'),
        current_price=current_price
    )

    # Determine portfolio action
    action = ta.determine_portfolio_action(
        signal=result['signal'],
        position_gap=position_gap,
        buy_quality=result.get('buy_quality', 'N/A')
    )

    # Count actions
    if action == "BUY":
        buy_count += 1
    elif action == "SELL":
        sell_count += 1
    else:
        hold_count += 1

    # Calculate buy tranches with individual quality ratings
    buy_tranches = ta.calculate_buy_tranches(
        gap_value=gap_value,
        s1=result.get('s1'),
        s2=result.get('s2'),
        s3=result.get('s3'),
        current_price=result['current_price'],
        s1_quality=(result.get('s1_quality', 'N/A'), result.get('s1_quality_note', '')),
        s2_quality=(result.get('s2_quality', 'N/A'), result.get('s2_quality_note', '')),
        s3_quality=(result.get('s3_quality', 'N/A'), result.get('s3_quality_note', ''))
    )

    # Calculate sell tranches with individual quality ratings
    sell_tranches = ta.calculate_sell_tranches(
        current_value=tradeable_value,
        signal=result['signal'],
        r1=result.get('r1'),
        r2=result.get('r2'),
        r3=result.get('r3'),
        adjusted_r1=adjusted_r1,
        adjusted_r2=adjusted_r2,
        adjusted_r3=adjusted_r3,
        r1_quality=(result.get('r1_quality', 'N/A'), result.get('r1_quality_note', '')),
        r2_quality=(result.get('r2_quality', 'N/A'), result.get('r2_quality_note', '')),
        r3_quality=(result.get('r3_quality', 'N/A'), result.get('r3_quality_note', ''))
    )

    # Build position dict
    position = {
        'ticker': ticker,
        'signal': result['signal'],
        'current_price': result['current_price'],
        'quantity': quantity,
        'min_quantity': min_quantity,
        'tradeable_quantity': tradeable_quantity,
        'current_value': current_value,
        'tradeable_value': tradeable_value,
        'min_value': min_value,
        'target_pct': target_pct,
        'current_pct': current_pct,
        'gap_value': gap_value,
        'action': action,
        'buy_quality': result.get('buy_quality', 'N/A'),
        'buy_quality_note': result.get('buy_quality_note', ''),
        's1_quality': result.get('s1_quality', 'N/A'),
        's1_quality_note': result.get('s1_quality_note', ''),
        's2_quality': result.get('s2_quality', 'N/A'),
        's2_quality_note': result.get('s2_quality_note', ''),
        's3_quality': result.get('s3_quality', 'N/A'),
        's3_quality_note': result.get('s3_quality_note', ''),
        'r1_quality': result.get('r1_quality', 'N/A'),
        'r1_quality_note': result.get('r1_quality_note', ''),
        'r2_quality': result.get('r2_quality', 'N/A'),
        'r2_quality_note': result.get('r2_quality_note', ''),
        'r3_quality': result.get('r3_quality', 'N/A'),
        'r3_quality_note': result.get('r3_quality_note', ''),
        'buy_tranches': buy_tranches,
        'sell_tranches': sell_tranches,
        'sell_feasibility_note': ma_note,
        # Technical levels for Technical Levels tab
        'd50': result.get('d50'),
        'd100': result.get('d100'),
        'd200': result.get('d200'),
        's1': result.get('s1'),
        's2': result.get('s2'),
        's3': result.get('s3'),
        'r1': result.get('r1'),
        'r2': result.get('r2'),
        'r3': result.get('r3'),
        # Volume Profile (VRVP) data
        'poc_60d': result.get('poc_60d'),
        'vah_60d': result.get('vah_60d'),
        'val_60d': result.get('val_60d'),
        'hvn_above_60d': result.get('hvn_above_60d'),
        'hvn_below_60d': result.get('hvn_below_60d'),
        'lvn_above_60d': result.get('lvn_above_60d'),
        'lvn_below_60d': result.get('lvn_below_60d'),
        'poc_52w': result.get('poc_52w'),
        'vah_52w': result.get('vah_52w'),
        'val_52w': result.get('val_52w')
    }

    portfolio_positions.append(position)

total_portfolio_value = PORTFOLIO_TOTAL + CASH_AVAILABLE
cash_pct_corrected = (CASH_AVAILABLE / total_portfolio_value) * 100

# Build portfolio_data structure expected by reporting functions
portfolio_data = {
    'positions': portfolio_positions,
    'portfolio_total': PORTFOLIO_TOTAL,
    'summary': {
        'buy_count': buy_count,
        'sell_count': sell_count,
        'hold_count': hold_count,
        'total_portfolio_value': total_portfolio_value,
        'cash_available': CASH_AVAILABLE
    }
}

print(f"\nPortfolio Summary:")
print(f"  Total Value: ${total_portfolio_value:,.2f}")
print(f"  Current Holdings: ${PORTFOLIO_TOTAL:,.2f}")
print(f"  Cash Available: ${CASH_AVAILABLE:,.2f} ({cash_pct_corrected:.1f}%)")
print(f"\nAction Summary:")
print(f"  BUY signals: {buy_count}")
print(f"  SELL signals: {sell_count}")
print(f"  HOLD signals: {hold_count}")



Portfolio Summary:
  Total Value: $141,869.27
  Current Holdings: $31,429.41
  Cash Available: $110,439.86 (77.8%)

Action Summary:
  BUY signals: 1
  SELL signals: 0
  HOLD signals: 6


In [14]:
# ===================================================================================================
# CELL 6: GENERATE REPORTS
# ===================================================================================================

# Generate Trading Playbook PDF
print("\n=== Generating Trading Playbook PDF ===")
pdf_path = RESULTS_DIR / f'trading_playbook_{TIMESTAMP}.pdf'
create_trading_playbook_pdf(portfolio_data, pdf_path, TIMESTAMP)
print(f"✓ Created: {pdf_path.name}")

# Generate Portfolio Tracker Excel
print("\n=== Generating Portfolio Tracker Excel ===")
excel_path = RESULTS_DIR / f'portfolio_tracker_{TIMESTAMP}.xlsx'
create_portfolio_tracker_excel(portfolio_data, excel_path)
print(f"✓ Created: {excel_path.name}")

# Archive old reports
print("\n=== Cleaning Up Old Reports ===")
cleanup_old_reports(RESULTS_DIR, max_files=1)

print(f"\n{'='*80}")
print("✅ PORTFOLIO ANALYSIS COMPLETE")
print(f"{'='*80}")
print(f"\nGenerated Files:")
print(f"  1. {pdf_path.name}")
print(f"  2. {excel_path.name}")
print(f"\nLocation: {RESULTS_DIR.absolute()}")



=== Generating Trading Playbook PDF ===
✓ Created: trading_playbook_20260108_185202.pdf

=== Generating Portfolio Tracker Excel ===
✓ Created: portfolio_tracker_20260108_185202.xlsx

=== Cleaning Up Old Reports ===
  📦 Archived: trading_playbook_20260108_184906.pdf
  📦 Archived: portfolio_tracker_20260108_184906.xlsx
  ✅ Archived 2 file(s), kept 1 most recent

✅ PORTFOLIO ANALYSIS COMPLETE

Generated Files:
  1. trading_playbook_20260108_185202.pdf
  2. portfolio_tracker_20260108_185202.xlsx

Location: c:\workspace\portfolio_analyser\portfolio_results
